In [25]:
import pandas as pd
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [26]:

import sys
sys.path.append('../utils')
import functions

In [27]:
df = pd.read_csv('../data/local/raw/doesthedogdie.com.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/local/raw/doesthedogdie.com.csv'

In [ ]:
functions.show_basic_info(df)


DataFrame Shape: (5910, 5)
Number of Rows: 5910
Number of Columns: 5

Data Types of Columns:
category_name     object
title             object
yes                int64
no                 int64
yes_ratio        float64
dtype: object

Missing Values per Column:
category_name    0
title            0
yes              0
no               0
yes_ratio        0
dtype: int64

First 5 Rows of Data:
  category_name                    title   yes   no  yes_ratio
0    a dog dies          Movie_John Wick  1150  106   0.915605
1    a dog dies        Movie_Marley & Me   781   37   0.954768
2    a dog dies         Movie_Hereditary   680   67   0.910308
3    a dog dies              Movie_MEGAN   580   22   0.963455
4    a dog dies  TV Show_Game of Thrones   524   36   0.935714


Check if there are duplicate titles

In [ ]:
duplicates = df[df['title'].duplicated()]

duplicate_count = df['title'].duplicated().sum()

print(f"Number of duplicates in 'title': {duplicate_count}")

Number of duplicates in 'title': 5176


### Check the categories in 'title' column

In [ ]:
# Extract the unique values before the underscore in 'title'
unique_titles = df['title'].str.split('_').str[0].unique()

print(unique_titles)

['Movie' 'TV Show' 'Video Game' 'Anime' 'Book' 'YouTube' 'Podcast' 'Manga'
 'Blog' 'Short Story']


Extract media type and remove it from title column

In [ ]:
df['media_type'] = df['title'].str.split('_').str[0]  # Extract media type
df['title'] = df['title'].str.split('_').str[1]       # Remove media type from title

df.head()

,category_name,title,yes,no,yes_ratio,media_type
0,a dog dies,John Wick,1150,106,0.915605,Movie
1,a dog dies,Marley & Me,781,37,0.954768,Movie
2,a dog dies,Hereditary,680,67,0.910308,Movie
3,a dog dies,MEGAN,580,22,0.963455,Movie
4,a dog dies,Game of Thrones,524,36,0.935714,TV Show


In [ ]:
print(df['media_type'].unique())

['Movie' 'TV Show' 'Video Game' 'Anime' 'Book' 'YouTube' 'Podcast' 'Manga'
 'Blog' 'Short Story']


#### Create new df for media_type 'Movie'

In [ ]:
doesthe_movies = df[df['media_type'] == 'Movie']
print(doesthe_movies)

             category_name                              title   yes   no  \
0               a dog dies                          John Wick  1150  106   
1               a dog dies                        Marley & Me   781   37   
2               a dog dies                         Hereditary   680   67   
3               a dog dies                              MEGAN   580   22   
5               a dog dies                       The Babadook   514   15   
...                    ...                                ...   ...  ...   
5898  there's gun violence                              Joker   104    2   
5901  there's gun violence                    American Psycho   101    0   
5906  there's gun violence  Everything Everywhere All at Once    92    2   
5907  there's gun violence                        Jojo Rabbit    92    0   
5909  there's gun violence                                Saw    91    1   

      yes_ratio media_type  
0      0.915605      Movie  
1      0.954768      Movie  


In [ ]:
doesthe_movies.to_csv('../data/local/raw/doesthedog_movies.csv', index=False)

In [ ]:
print(f'Unique titles in doesthe_movies: {doesthe_movies['title'].nunique()}')

Unique titles in doesthe_movies: 433


In [ ]:
doesthe_movies['clean_title'] = functions.prepare_clean_titles(doesthe_movies, 'title')
doesthe_movies.head()


/tmp/ipykernel_26246/2012065974.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doesthe_movies['clean_title'] = functions.prepare_clean_titles(doesthe_movies, 'title')


,category_name,title,yes,no,yes_ratio,media_type,clean_title
0,a dog dies,John Wick,1150,106,0.915605,Movie,john wick
1,a dog dies,Marley & Me,781,37,0.954768,Movie,marley me
2,a dog dies,Hereditary,680,67,0.910308,Movie,hereditary
3,a dog dies,MEGAN,580,22,0.963455,Movie,megan
5,a dog dies,The Babadook,514,15,0.971645,Movie,the babadook


In [ ]:
functions.show_basic_info(doesthe_movies)


DataFrame Shape: (2836, 7)
Number of Rows: 2836
Number of Columns: 7

Data Types of Columns:
category_name     object
title             object
yes                int64
no                 int64
yes_ratio        float64
media_type        object
clean_title       object
dtype: object

Missing Values per Column:
category_name    0
title            0
yes              0
no               0
yes_ratio        0
media_type       0
clean_title      0
dtype: int64

First 5 Rows of Data:
  category_name         title   yes   no  yes_ratio media_type   clean_title
0    a dog dies     John Wick  1150  106   0.915605      Movie     john wick
1    a dog dies   Marley & Me   781   37   0.954768      Movie    marley  me
2    a dog dies    Hereditary   680   67   0.910308      Movie    hereditary
3    a dog dies         MEGAN   580   22   0.963455      Movie         megan
5    a dog dies  The Babadook   514   15   0.971645      Movie  the babadook


Drop media_type column

In [ ]:
doesthe_movies.drop(columns=['media_type'], inplace=True)
doesthe_movies.head()

/tmp/ipykernel_26246/1615881242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  doesthe_movies.drop(columns=['media_type'], inplace=True)


,category_name,title,yes,no,yes_ratio,clean_title
0,a dog dies,John Wick,1150,106,0.915605,john wick
1,a dog dies,Marley & Me,781,37,0.954768,marley me
2,a dog dies,Hereditary,680,67,0.910308,hereditary
3,a dog dies,MEGAN,580,22,0.963455,megan
5,a dog dies,The Babadook,514,15,0.971645,the babadook


#### Extract category_name unique values

In [ ]:
# Extract unique values, sort them, and print the list
unique_values = sorted(doesthe_movies['category_name'].dropna().unique())
print(unique_values)

["A child's dear toy is destroyed", 'Autism specific abuse', 'D.I.D. Misrepresentation', 'Santa (et al) is spoiled', 'Someone attempts suicide', 'Someone becomes unconscious', "There's audio gore", 'a baby cries', 'a baby is stillborn', 'a car crashes', 'a car honks or tires screech', 'a cat dies', 'a child is abandoned by a parent or guardian', 'a dog dies', 'a dragon dies', 'a family member dies', 'a horse dies', 'a kid dies', 'a major character dies', 'a male character is ridiculed for crying', 'a mentally ill person is violent', 'a minor is sexualized', 'a minority is misrepresented', 'a non-human character dies', 'a parent dies', 'a person is hit by a car', 'a pet dies', 'a plane crashes', 'a pregnant person dies', 'a priceless artifact is destroyed', 'a trans person is depicted predatorily', 'a woman gets slapped', 'a woman is brutalized for spectacle', 'alcohol abuse', 'an LGBT person dies', 'an LGBT+ person is outed', 'an animal dies', 'an animal is abandoned', 'an animal is sa

### Drop rows where 'yes' is greater than 'no'
Keep rows where sensitive content does take place.

In [ ]:
doesthe_movies = doesthe_movies[doesthe_movies['yes'] <= doesthe_movies['no']]

In [ ]:
# functions.show_basic_info(doesthe_movies)


DataFrame Shape: (534, 6)
Number of Rows: 534
Number of Columns: 6

Data Types of Columns:
category_name     object
title             object
yes                int64
no                 int64
yes_ratio        float64
clean_title       object
dtype: object

Missing Values per Column:
category_name    0
title            0
yes              0
no               0
yes_ratio        0
clean_title      0
dtype: int64

First 5 Rows of Data:
           category_name            title  yes   no  yes_ratio  \
600  there's child abuse           Barbie   16  299   0.050794   
601  there's child abuse        Midsommar  142  266   0.348039   
602  there's child abuse             Nope   29  191   0.131818   
603  there's child abuse       Knives Out    1  150   0.006623   
604  there's child abuse  American Psycho    0  136   0.000000   

         clean_title  
600           barbie  
601        midsommar  
602             nope  
603       knives out  
604  american psycho  


In [ ]:
print(f'Unique titles in doesthe_movies: {doesthe_movies['title'].nunique()}')

Unique titles in doesthe_movies: 89


In [ ]:
# Group by 'clean_title' and concatenate 'category_name' with a comma
events_df = doesthe_movies.groupby('clean_title')['category_name'].apply(lambda x: ', '.join(x)).reset_index()

# Rename the column to 'events'
events_df.rename(columns={'category_name': 'events'}, inplace=True)

# Merge the aggregated events back into the original DataFrame
doesthe_movies = pd.merge(doesthe_movies, events_df, on='clean_title', how='left')

In [ ]:
events_df.head(100)

,clean_title,events
0,a quiet place,"there's child abuse, there's domestic violence..."
1,a quiet place day one,"a woman is brutalized for spectacle, someone i..."
2,abigail,there's BDSM
3,alien,"there's child abuse, there's domestic violence..."
4,alien romulus,"a woman is brutalized for spectacle, someone i..."
...,...,...
84,titanic,there's child abuse
85,us,"there's child abuse, there's domestic violence..."
86,venom,someone dies by suicide
87,wonka,"a woman is brutalized for spectacle, a woman g..."
